In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel

In [2]:
# Load data
data = pd.read_csv('sensor_data.csv')

In [55]:
data.count()

ID             6583
timestamp      6583
latitude       6583
longitude      6583
temperature    6345
humidity       6321
dtype: int64

In [12]:
#preparing data for temperature mode
temp_data = data.copy()
temp_data.dropna(subset=['temperature'],inplace = True)
temp_data.interpolate(inplace=True)

In [13]:
temp_data.count()

ID             6345
timestamp      6345
latitude       6345
longitude      6345
temperature    6345
humidity       6345
dtype: int64

In [18]:
train_temp, test_temp = train_test_split(temp_data, test_size=0.2, random_state=42)

In [19]:
train_temp.count()

ID             5076
timestamp      5076
latitude       5076
longitude      5076
temperature    5076
humidity       5076
dtype: int64

In [21]:
test_temp.count()

ID             1269
timestamp      1269
latitude       1269
longitude      1269
temperature    1269
humidity       1269
dtype: int64

In [23]:
train_X_temp = train_temp[['latitude', 'longitude', 'humidity', 'timestamp']]
train_y_temp = train_temp['temperature']

In [25]:
test_X_temp = test_temp[['latitude', 'longitude', 'humidity', 'timestamp']]
test_y_temp = test_temp['temperature']

In [73]:
# Define kernel functions
kernel_temp = ConstantKernel(1.0) * RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0)

In [74]:
model_temp = GaussianProcessRegressor(kernel=kernel_temp)
model_temp.fit(train_X_temp, train_y_temp)

/Users/inavin/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1))

In [75]:
pred_y_temp = model_temp.predict(test_X_temp)

In [76]:
pred_y_temp

array([28.93691882, 20.25387284, 16.50950939, ..., 22.92377963,
       17.28025256, 17.21877995])

In [80]:
# Evaluate model on test data
test_rmse_temp = ((test_y_temp - pred_y_temp)**2).mean()**0.5
print(test_rmse_temp)


0.8378323618098715


In [56]:
#preparing data for temperature mode
humid_data = data.copy()
humid_data.dropna(subset=['humidity'],inplace = True)
humid_data.interpolate(inplace=True)

In [57]:
humid_data.count()

ID             6321
timestamp      6321
latitude       6321
longitude      6321
temperature    6321
humidity       6321
dtype: int64

In [58]:
train_humid, test_humid = train_test_split(humid_data, test_size=0.2, random_state=42)

In [59]:
train_humid.count()

ID             5056
timestamp      5056
latitude       5056
longitude      5056
temperature    5056
humidity       5056
dtype: int64

In [60]:
test_humid.count()

ID             1265
timestamp      1265
latitude       1265
longitude      1265
temperature    1265
humidity       1265
dtype: int64

In [61]:
train_X_humid = train_humid[['latitude', 'longitude', 'temperature', 'timestamp']]
train_y_humid = train_humid['humidity']

In [62]:
test_X_humid = test_humid[['latitude', 'longitude', 'temperature', 'timestamp']]
test_y_humid = test_humid['humidity']

In [66]:
# Define kernel functions
kernel_humid = ConstantKernel(1.0) * RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0)


In [67]:
model_humid = GaussianProcessRegressor(kernel=kernel_humid)
model_humid.fit(train_X_humid, train_y_humid)

GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1))

In [68]:
pred_y_humid = model_humid.predict(test_X_humid)

In [69]:
test_rmse_temp = ((test_y_humid - pred_y_humid)**2).mean()**0.5

In [70]:
print(test_rmse_temp)

4.269382978090558
